In [1]:
import pandas as pd
df_eng = pd.read_csv('takehome_user_engagement.csv',parse_dates=[0])

/Anaconda/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


The encoding in takehome_users.csv must be detected to read it. This is done with chardet.

In [2]:
import chardet
chardet.detect(open('takehome_users.csv','rb').read())

{'confidence': 0.7299208911655781, 'encoding': 'windows-1252'}

In [3]:
df_user = pd.read_csv('takehome_users.csv', encoding='windows-1252',index_col='object_id')

In [4]:
df_user.head(5)

,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
object_id,,,,,,,,,
1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [5]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 1 to 12000
Data columns (total 9 columns):
creation_time                 12000 non-null object
name                          12000 non-null object
email                         12000 non-null object
creation_source               12000 non-null object
last_session_creation_time    8823 non-null float64
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            6417 non-null float64
dtypes: float64(2), int64(3), object(4)
memory usage: 937.5+ KB


In [6]:
df_eng.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [7]:
df_eng.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
time_stamp    207917 non-null datetime64[ns]
user_id       207917 non-null int64
visited       207917 non-null int64
dtypes: datetime64[ns](1), int64(2)
memory usage: 4.8 MB


In [8]:
from datetime import datetime, timedelta

def custom(x):
    """
    Takes property object x
    converts x to list
    returns 1 if a user logged into the product on three separate
    days in at least one seven day period 
    """
    if len(x) >= 3:  # condition to eliminate if user has less than 3 time stamps
        x = [i for i in x]  # convert property object to list
        x.sort()  # sort the dates in increasing order
        x = [x[i+1] - x[i] for i in range(len(x)-2)]  # compute cumulative difference of current and next day
        # sum i, i+1, i+2 terms and check if it`s less than 7 which gives if it`s in a seven day period
        x = [1 for i in range(len(x)-2) if x[i] + x[i+1] + x[i+2] <= timedelta(days=7)]
        # condition to check if there`s a 1 in x, return 1 if true
        if 1 in x:
            return 1


Applying the custom function to df_eng to find out the engaged users

In [9]:
df1 = df_eng.groupby('user_id').agg(custom)

Fill Nan values with 0.

In [10]:
df1.fillna(0, inplace=True)
df1.columns = ['adopted_user']

In [11]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8823 entries, 1 to 12000
Data columns (total 1 columns):
adopted_user    8823 non-null float64
dtypes: float64(1)
memory usage: 137.9 KB


In [12]:
n_ad = df1['adopted_user'].value_counts()

In [13]:
n_ad

0.0    7501
1.0    1322
Name: adopted_user, dtype: int64

Out of 8823 users, 1322 users are 'engaged'.

In [14]:
 df_user = df_user.drop(['name','email'], axis=1)

User name and email were not considered as dependent variables!

In [15]:
df_user.head()

,creation_time,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
object_id,,,,,,,
1,2014-04-22 03:53:30,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
2,2013-11-15 03:45:04,ORG_INVITE,1.396238e+09,0,0,1,316.0
3,2013-03-19 23:14:52,ORG_INVITE,1.363735e+09,0,0,94,1525.0
4,2013-05-21 08:09:28,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
5,2013-01-17 10:14:20,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [16]:
df_user['creation_source'].unique()

array(['GUEST_INVITE', 'ORG_INVITE', 'SIGNUP', 'PERSONAL_PROJECTS',
       'SIGNUP_GOOGLE_AUTH'], dtype=object)

The unique creation sources considered as variables. The variables were encoded as dummy variables.

In [17]:
creation_source = pd.get_dummies(df_user['creation_source'])

In [18]:
invited_user = df_user['invited_by_user_id'].copy()

In [19]:
invited_user.fillna(0, inplace=True)

The user was encoded as 1 if invited as 1 and 0 if not Nan i.e. if not invited by anyone.

In [20]:
def rep_one(x):
    if x != 0:
        return 1
    else:
        return 0
invited_user = invited_user.apply(rep_one)

In [21]:
invited_user.head()

object_id
1    1
2    1
3    1
4    1
5    1
Name: invited_by_user_id, dtype: int64

In [22]:
creation_time = df_user['creation_time'].copy()

The difference between creation date and last usage date was taken as length of engagement or usage of the account.

In [23]:
import numpy as np
creation_time = pd.to_datetime(creation_time)
creation_time = creation_time.astype(np.int64)//10**9

In [24]:
last_session = df_user['last_session_creation_time'].copy()

In [25]:
usage = last_session-creation_time

In [26]:
usage.fillna(0,inplace = True)

In [27]:
df_user.columns

Index(['creation_time', 'creation_source', 'last_session_creation_time',
       'opted_in_to_mailing_list', 'enabled_for_marketing_drip', 'org_id',
       'invited_by_user_id'],
      dtype='object')

The irrelevant columns were dropped from the df_user dataframe and the relevant calculated columns were added to create DataFrame df_user1

In [28]:
df_user1 = df_user.drop(['creation_time', 'creation_source', 'last_session_creation_time','invited_by_user_id'], axis=1)

In [29]:
df_user1 = df_user1.join(creation_source)

In [30]:
df_user1['invited'] = invited_user

In [31]:
df_user1['usage'] = usage

In [32]:
df_user1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 1 to 12000
Data columns (total 10 columns):
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
GUEST_INVITE                  12000 non-null uint8
ORG_INVITE                    12000 non-null uint8
PERSONAL_PROJECTS             12000 non-null uint8
SIGNUP                        12000 non-null uint8
SIGNUP_GOOGLE_AUTH            12000 non-null uint8
invited                       12000 non-null int64
usage                         12000 non-null float64
dtypes: float64(1), int64(4), uint8(5)
memory usage: 941.1 KB


In [33]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8823 entries, 1 to 12000
Data columns (total 1 columns):
adopted_user    8823 non-null float64
dtypes: float64(1)
memory usage: 137.9 KB


The dependent variable dataframe df1 has fewer columns than independent variables dataframe. The idex of the df1 were matched with those of df_user1. 

In [34]:
idx = df1.index

In [35]:
X = df_user1.loc[idx,:]

In [36]:
y = df1

In [37]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler

/Anaconda/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Anaconda/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Anaconda/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Anaconda/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [38]:
scaler = StandardScaler()
scaled_X = scaler.fit_transform(X)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.33, random_state=42)

In [40]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [41]:
y_pred = clf.predict(X_test)

In [42]:
print('=====Feature importance=======')
for idx, val in enumerate(clf.feature_importances_):
    print("{:30}{:3f}".format(X.columns[idx], val))
print('-----------------------------------------------')

print("{:30}{:3f}".format('F1 score', f1_score(y_test, y_pred)))
print("{:30}{:3f}".format('Test accuracy', precision_score(y_test, y_pred)))
print("{:30}{:3f}".format('Test recall', recall_score(y_test, y_pred)))
print("{:30}{:3f}".format('Test precision', accuracy_score(y_test, y_pred)))
print()

print('====Confusion Matrix=====')
print(confusion_matrix(y_test, y_pred))

=====Feature importance=======
opted_in_to_mailing_list      0.006480
enabled_for_marketing_drip    0.008553
org_id                        0.071189
GUEST_INVITE                  0.004218
ORG_INVITE                    0.005419
PERSONAL_PROJECTS             0.000669
SIGNUP                        0.003771
SIGNUP_GOOGLE_AUTH            0.001740
invited                       0.004847
usage                         0.893115
-----------------------------------------------
F1 score                      0.859375
Test accuracy                 0.849890
Test recall                   0.869074
Test precision                0.956731

====Confusion Matrix=====
[[2401   68]
 [  58  385]]


The 'usage' feature was most important with importance score of 0.89.

In [43]:
new_X = X.drop(['usage'], axis=1)

In [45]:
scaler = StandardScaler()
scaled_new_X = scaler.fit_transform(new_X)
X_train, X_test, y_train, y_test = train_test_split(scaled_new_X, y, test_size=0.33, random_state=42)
clf1 = DecisionTreeClassifier()
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)

In [46]:
print('=====Feature importance=======')
for idx, val in enumerate(clf1.feature_importances_):
    print("{:30}{:3f}".format(new_X.columns[idx], val))
print('-----------------------------------------------')

print("{:30}{:3f}".format('F1 score', f1_score(y_test, y_pred)))
print("{:30}{:3f}".format('Test accuracy', precision_score(y_test, y_pred)))
print("{:30}{:3f}".format('Test recall', recall_score(y_test, y_pred)))
print("{:30}{:3f}".format('Test precision', accuracy_score(y_test, y_pred)))
print()

print('====Confusion Matrix=====')
print(confusion_matrix(y_test, y_pred))

=====Feature importance=======
opted_in_to_mailing_list      0.080684
enabled_for_marketing_drip    0.067455
org_id                        0.728656
GUEST_INVITE                  0.011352
ORG_INVITE                    0.023269
PERSONAL_PROJECTS             0.015448
SIGNUP                        0.018890
SIGNUP_GOOGLE_AUTH            0.034531
invited                       0.019714
-----------------------------------------------
F1 score                      0.148148
Test accuracy                 0.200772
Test recall                   0.117381
Test precision                0.794643

====Confusion Matrix=====
[[2262  207]
 [ 391   52]]


After usage, org id was important feature. However, the model performed much better with the usage feature than without it.